In [ ]:
import MDAnalysis as mda
import numpy as np
from tqdm import tqdm
import warnings
import os
import gc
from distutils.dir_util import copy_tree
warnings.filterwarnings('ignore')

In [ ]:
def make_tether(_u, _PEG, _cpt_res, _ns, start, stop):
    _cpt_res = cpt_res
    _PEG = PEG
    _u = u
    _allx = []
    _ally = []
    _allz = []
    for _ts in _u.trajectory[start:stop]:
        _allx.append(_PEG.atoms.positions.T[0])
        _ally.append(_PEG.atoms.positions.T[1])
        _allz.append(_PEG.atoms.positions.T[2])
    f = open('tether/PEG'+str(_cpt_res+1)+'/ns'+str(_ns)+'/TETHER.x.lammps', 'w')
    for x in _allx:
        f.write(str(np.round(x[0],2))+' '+str(np.round(x[1],2))+' '+str(np.round(x[2],2))+' '+str(np.round(x[3],2))+' '+str(np.round(x[4],2))+'\n')
    f.close()
    f = open('tether/PEG'+str(_cpt_res+1)+'/ns'+str(_ns)+'/TETHER.y.lammps', 'w')
    for y in _ally:
        f.write(str(np.round(y[0],2))+' '+str(np.round(y[1],2))+' '+str(np.round(y[2],2))+' '+str(np.round(y[3],2))+' '+str(np.round(y[4],2))+'\n')
    f.close()
    f = open('tether/PEG'+str(_cpt_res+1)+'/ns'+str(_ns)+'/TETHER.z.lammps', 'w')
    for z in _allz:
        f.write(str(np.round(z[0],2))+' '+str(np.round(z[1],2))+' '+str(np.round(z[2],2))+' '+str(np.round(z[3],2))+' '+str(np.round(z[4],2))+'\n')
    f.close()

In [ ]:
path1 = "C-CG/"

In [ ]:
if not os.path.exists('dump/'):
    os.mkdir('dump')
original_univers = mda.Universe(path1+"conf.gro", path1+"traj.trr")
dimensions = original_univers.dimensions
n_frames = original_univers.trajectory.n_frames
#original_univers.transfer_to_memory(stop=n_frames)

In [ ]:

timestep = np.round(original_univers.trajectory.dt,2)
f = open("dump/dimensions.dat", "w")
f.write(str(dimensions[0])+' '+str(dimensions[1])+' '+str(dimensions[2])+' '+str(dimensions[3])+' '+str(dimensions[4])+' '+str(dimensions[5])+'\n')
f.close()
dimensions = np.loadtxt("dump/dimensions.dat")

In [ ]:
# make peg list
PEG_list = []
for cpt_res, ids in enumerate(original_univers.residues.ids):
    PEG_list.append(cpt_res)

In [ ]:
# make frame list
frames = np.arange(n_frames)
frame_list = np.array_split(frames, 100)

In [ ]:
dimensions = original_univers.dimensions
f = open("dump/dimensions.dat", "w")
f.write(str(dimensions[0])+' '+str(dimensions[1])+' '+str(dimensions[2])+' '+str(dimensions[3])+' '+str(dimensions[4])+' '+str(dimensions[5])+'\n')
f.close()
for ts in original_univers.trajectory[::100]:
    assert dimensions[0] == ts.dimensions[0] # volume of box changed
    assert dimensions[1] == ts.dimensions[1] # volume of box changed
    assert dimensions[2] == ts.dimensions[2] # volume of box changed

In [ ]:
if not os.path.exists('data/'):
    os.mkdir('data')

In [ ]:
# create group
original_groups = [[1,2,3,4,5,],
                   [6,7,8,9,10,11,12],
                   [13,14,15,16,17,18,19],
                   [20,21,22,23,24,25,26],
                   [27,28,29,30,31]]
f = open("inputs/GROUP.lammps", "w")
for cpt_grp, group in enumerate(original_groups):
    line = 'group MON'+str(cpt_grp+1)+' id'
    for atom in group:
        line = line+' '+str(atom)
    f.write(line+'\n')
f.close()

In [ ]:
f = open('PEG.list', 'w')
for x in PEG_list:
    f.write(str(x)+'\n')
f.close()

In [ ]:
# create one PEG folder per PEG
for cpt_res in PEG_list:
    if not os.path.exists('dump/PEG'+str(cpt_res+1)):
        os.mkdir('dump/PEG'+str(cpt_res+1))
        #ns = 0
        #if not os.path.exists('dump/PEG'+str(cpt_res+1)+'/ns'+str(ns)):
        #    os.mkdir('dump/PEG'+str(cpt_res+1)+'/ns'+str(ns))

In [ ]:
# generate tether positions from CG trajectories
#if not os.path.exists('tether/'):
#    os.mkdir('tether')
#for cpt_res in PEG_list:
#    if not os.path.exists('tether/PEG'+str(cpt_res+1)):
#        os.mkdir('tether/PEG'+str(cpt_res+1))
#        print('PEG'+str(cpt_res+1))
#        #for ns, frame in enumerate(frame_list):
#        ns = 0
#        frame = 0
#        #print('Dealing with ns'+str(ns))
#        #os.mkdir('tether/PEG'+str(cpt_res+1)+'/ns'+str(ns))
#        u = mda.Universe(path1+"conf.gro", path1+"traj.xtc")
##        #u.transfer_to_memory(stop=n_frames)
#        #u.transfer_to_memory(start = frame[0], stop = frame[-1]+1)
#        PEG = u.select_atoms('resindex '+str(cpt_res))
#        make_tether2(u, PEG, cpt_res)

In [ ]:
if not os.path.exists('tether/'):
    os.mkdir('tether')
for cpt_res in PEG_list:
    if not os.path.exists('tether/PEG'+str(cpt_res+1)):
        os.mkdir('tether/PEG'+str(cpt_res+1))
        print('PEG'+str(cpt_res+1))
        u = mda.Universe(path1+"conf.gro", path1+"traj.trr")
        #u.transfer_to_memory()
        PEG = u.select_atoms('resindex '+str(cpt_res))
        for ns, frame in enumerate(frame_list):
            if not os.path.exists('tether/PEG'+str(cpt_res+1)+'/ns'+str(ns)):
                os.mkdir('tether/PEG'+str(cpt_res+1)+'/ns'+str(ns))
                make_tether(u, PEG, cpt_res, ns, frame[0], frame[-1]+1)
        gc.collect()